In [ ]:
import utils
import plotting
import postprocessing
import corrections
from collections import OrderedDict

from utils import CUT_MAX_VAL
from hh_vars import (
    years,
    data_key,
    qcd_key,
    bg_keys,
    samples,
    nonres_sig_keys,
    nonres_samples,
    txbb_wps,
    jec_shifts,
    jmsr_shifts,
)
from postprocessing import nonres_shape_vars, Region

import numpy as np
import pandas as pd
import pickle

# from pandas.errors import SettingWithCopyWarning
from hist import Hist

import os
from copy import deepcopy
from inspect import cleandoc
import warnings

import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib.ticker as mticker

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))
plt.rcParams.update({"font.size": 16})

# ignore these because they don't seem to apply
# warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
MAIN_DIR = "../../../"
samples_dir = f"{MAIN_DIR}/../data/skimmer/anava/Test"
# samples_dir = "/eos/uscms/store/user/anava/bbVV/skimmer/Test/"
year = "2017"

date = "23Sep2"
plot_dir = f"../../../plots/PostProcessing/{date}/"
templates_dir = f"templates/{date}"
_ = os.system(f"mkdir -p {plot_dir}")
_ = os.system(f"mkdir -p {plot_dir}/cutflows/")
_ = os.system(f"mkdir -p {plot_dir}/ControlPlots/{year}/")
_ = os.system(f"mkdir -p {plot_dir}/templates/")
# _ = os.system(f"mkdir -p {plot_dir}/templates/wshifts")
# _ = os.system(f"mkdir -p {plot_dir}/templates/jshifts")
# _ = os.system(f"mkdir -p {templates_dir}")

Load samples

In [ ]:
filters = postprocessing.new_filters
systematics = {year: {}}

# save cutflow as pandas table
cutflow = pd.DataFrame(index=list(samples.keys()) + list(nonres_samples.keys()))

# utils.remove_empty_parquets(samples_dir, year)
events_dict = utils.load_samples(samples_dir, nonres_samples, year, filters)
events_dict |= utils.load_samples(samples_dir, samples, year, filters)

utils.add_to_cutflow(events_dict, "Preselection", "weight", cutflow)

cutflow

Scale factors and bb VV assignment

In [ ]:
postprocessing.apply_weights(events_dict, year, cutflow)
bb_masks = postprocessing.bb_VV_assignment(events_dict)
# events_dict[sig_key] = postprocessing.postprocess_lpsfs(events_dict[sig_key])
cutflow

Control plots

In [ ]:
# {var: (bins, label)}
control_plot_vars = {
    # "MET_pt": ([50, 0, 300], r"$p^{miss}_T$ (GeV)"),
    "DijetEta": ([30, -8, 8], r"$\eta^{jj}$"),
    "DijetPt": ([30, 0, 750], r"$p_T^{jj}$ (GeV)"),
    "DijetMass": (
        # list(range(800, 1400, 100)) + [1400, 1600, 2000, 3000, 4400],
        [30, 600, 4500],
        r"$m^{jj}$ (GeV)",
    ),
    "bbFatJetEta": ([30, -2.4, 2.4], r"$\eta^{bb}$"),
    "bbFatJetPt": ([30, 300, 1500], r"$p^{bb}_T$ (GeV)"),
    "bbFatJetParticleNetMass": ([20, 50, 250], r"$m^{bb}_{reg}$ (GeV)"),
    # "bbFatJetMsd": ([50, 0, 300], r"$m^{bb}_{msd}$ (GeV)"),
    # "bbFatJetParticleNetMD_Txbb": ([50, 0.8, 1], r"$T^{bb}_{Xbb}$"),
    "VVFatJetEta": ([30, -2.4, 2.4], r"$\eta^{VV}$"),
    "VVFatJetPt": ([30, 300, 1500], r"$p^{VV}_T$ (GeV)"),
    "VVFatJetParticleNetMass": (
        # list(range(50, 110, 10)) + list(range(110, 200, 15)) + [200, 220, 250],
        [20, 50, 250],
        r"$m^{VV}_{reg}$ (GeV)",
    ),
    # "VVFatJetMsd": ([40, 50, 250], r"$m^{VV}_{msd}$ (GeV)"),
    # "VVFatJetParticleNet_Th4q": ([50, 0, 1], r"Prob($H \to 4q$) vs Prob(QCD) (Non-MD)"),
    # "VVFatJetParTMD_THWW4q": (
    #     [50, 0, 1],
    #     r"Prob($H \to VV \to 4q$) vs Prob(QCD) (Mass-Decorrelated)",
    # ),
    # "VVFatJetParTMD_probT": ([50, 0, 1], r"Prob(Top) (Mass-Decorrelated)"),
    # "VVFatJetParTMD_THWWvsT": (
    #     [50, 0, 1],
    #     r"$T^{VV}_{HWW}$",
    # ),
    # "bbFatJetPtOverDijetPt": ([50, 0, 40], r"$p^{bb}_T / p_T^{jj}$"),
    # "VVFatJetPtOverDijetPt": ([50, 0, 40], r"$p^{VV}_T / p_T^{jj}$"),
    # "VVFatJetPtOverbbFatJetPt": ([50, 0.4, 2.0], r"$p^{VV}_T / p^{bb}_T$"),
    # "nGoodMuons": ([3, 0, 3], r"# of Muons"),
    # "nGoodElectrons": ([3, 0, 3], r"# of Electrons"),
    # "nGoodJets": ([5, 0, 5], r"# of AK4 B-Jets"),
    # "BDTScore": ([50, 0, 1], r"BDT Score"),
}

hists = postprocessing.control_plots(
    events_dict,
    bb_masks,
    ["HHbbVV", "VBFHHbbVV", "qqHH_CV_1_C2V_0_kl_1_HHbbVV", "qqHH_CV_1_C2V_2_kl_1_HHbbVV"],
    control_plot_vars,
    f"{plot_dir}/ControlPlots/{year}/",
    year,
    bg_keys=bg_keys,
    # bg_keys=["QCD", "TT", "ST", "V+Jets", "Hbb"],
    sig_scale_dict={
        "HHbbVV": 1e5,
        "VBFHHbbVV": 2e5,
        "qqHH_CV_1_C2V_0_kl_1_HHbbVV": 2e3,
        "qqHH_CV_1_C2V_2_kl_1_HHbbVV": 2e3,
    },
    show=True,
    log="both",
)

Templates

In [ ]:
# add actual cuts here
selection_regions = {
    "pass": postprocessing.Region(
        cuts={
            "bbFatJetParticleNetMD_Txbb": [0.99, CUT_MAX_VAL],
        },
        label="Pass",
    ),
    "fail": postprocessing.Region(
        cuts={
            "bbFatJetParticleNetMD_Txbb": [-CUT_MAX_VAL, 0.99],
        },
        label="Fail",
    ),
}

# variable to plot
shape_vars = [
    postprocessing.ShapeVar(
        "bbFatJetParticleNetMass",
        r"$m^{bb}_\mathrm{Reg}$ (GeV)",
        [20, 50, 250],
        reg=True,
        blind_window=[100, 150],
    ),
]

h, tsysts = postprocessing.get_templates(
    events_dict,
    bb_masks,
    year,
    nonres_sig_keys,
    selection_regions,
    shape_vars,
    systematics,
    bg_keys=bg_keys,
    plot_sig_keys=[
        "HHbbVV",
        "VBFHHbbVV",
        "qqHH_CV_1_C2V_0_kl_1_HHbbVV",
        "qqHH_CV_1_C2V_2_kl_1_HHbbVV",
    ],
    sig_scale_dict={
        "HHbbVV": 1e4,
        "VBFHHbbVV": 1e5,
        "qqHH_CV_1_C2V_0_kl_1_HHbbVV": 2e2,
        "qqHH_CV_1_C2V_2_kl_1_HHbbVV": 2e2,
    },
    plot_dir=f"{plot_dir}/templates",
    prev_cutflow=cutflow,
    weight_shifts={},
    jshift="",
    plot_shifts=False,
    lpsfs=False,
    show=True,
)